<a href="https://colab.research.google.com/github/okayhrm/ARIMAtimeseries/blob/main/Timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']= 10,6

In [ ]:
dataset = pd.read_excel('')
medname = dataset[dataset['Medicine']=='Name']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
plt.xlabel('Medicines Sold')
plt.ylabel('Month')
plt.plot(medname)

In [ ]:
#determining singularity
rolmean = medname.rolling.mean()
rolstd = medname.rolling.std()
print(rolmean,rolstd)

In [ ]:
#plotting it
original = plt.plot(medname,color = 'blue',label = 'original')
mean = plt.plot(rolmean,color = 'red', label = 'Rolling Mean')
std = plt.plot(rolstd, color = 'black', label = 'Rolling Std')
plt.title('Rolling Mean and Standard Deviation')
plt.show(block=false)


In [ ]:
#Augumented Dickey - Fuler Test(Do either one , both for practice)
from statsmodels.tsa.stattools import adfuller
print('Results of Dickey Fuller Test:')
dftest = adfuller(medname['Consumption'], autolag = 'AIC')
dfoutput = pd.series(dftest[0:4], index =['Test Statistc','p-value','Lags used','number of observations used'])
for key,value in dftest[4].items():
  dfoutput['Criticial Value (%s)'%key] = value
print(dfoutput)


In [ ]:
#done to create singularity
medname_logscale = np.log(medname)
plt.plot(medname_logscale)

In [ ]:
movingaverage = medname_logscale.rolling.mean()
movingstd = medname_logscale.rolling.std()
plt.plot(medname_logscale)
plt.plot(movingaverage, color = 'red')

In [ ]:
#LMMA = data set log set minus moving average
LLMA = medname_logscale - movingaverage

LLMA.dropna(inplace=True)

In [ ]:
from statsmodels.tsa.stattools
def test_stationary(timeseries):
  movingavg = timeseries.rolling.mean()
  movingSTD = timeseries.rolling.std()

  orig = plt.plot(timeseries, color = 'blue',label = 'original')
  mean = plt.plot(movingavg, color ='red',label = 'Rolling Mean')
  std = plt.plot(movingSTD,color = 'black', label = 'Rolling std')
  plt.legend(loc = 'best')
  plt.title('Rolling Mean and Rolling Std')
  plt.show(block =False)

  dftest = adfuller(timeseries['Consumption'], autolag ='AIC')
  dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Obeservations'])
  for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)',%key] = value
  print(dfoutput)




In [ ]:
test_stationary(LLMA)

In [ ]:
#DSLDS = Data set logdiff shift
DSLDS = medname_logscale - medname_logscale.shift()
plt.plot(DSLDS)

In [ ]:
DSLDS.dropna()

In [ ]:
test_stationary(DSLDS)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(medname_logscale)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.residual

plt.subplot(411)
plt.plot(medname_logscale,label = 'original')
plt.subplot(412)
plt.plot(trend,label = 'trend')
plt.subplot(413)
plt.plot(seasonal, label = 'seasonality')
plt.subplot(414)
plt.plot(residual, label = 'residual')
plt.tight_layout()

decomposedLogdata = residual
decomposedLogdata.dropna()
test_stationary(decomposedLogdata)


In [ ]:

# to find p and q value we plot pcaf and acf graph
from statsmodels.tsa.stattools import acf,pacf

lag_acf = acf(DSLDS, nlags=20)
lag_pacf = pacf(DSLDS, nlags=20)

plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y = 0, linestyle ="--", color = 'gray')
plt.axhline(y =1.96/np.sqrt(len(DSLDS)), linestyle = "--", color = 'gray' )
plt.axhline(y=1.96/np.sqrt(len(DSLDS)),linestyle = "--",color = 'gray')
plt.title('Autocorrelation Function')

plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y = 0, linestyle ="--", color = 'gray')
plt.axhline(y =1.96/np.sqrt(len(DSLDS)), linestyle = "--", color ='gray' )
plt.axhline(y=1.96/np.sqrt(len(DSLDS)),linestyle = "--",color = 'gray')
plt.title('Partial Autocorrelation Function')

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(medname_logscale, order =(2,1,0))
results_AR = model.fit(disp = -1)
plt.plot(DSLDS)
plt.plot(results_AR.fittedvalues, color ='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-DSLDS['Consumption'])**2))
print('Plotting AR model')


In [ ]:
model = ARIMA(medname_logscale, order =(0,1,2))
results_MA = model.fit(disp = -1)
plt.plot(DSLDS)
plt.plot(results_MA.fittedvalues, color ='red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-DSLDS['Consumption'])**2))
print('Plotting MA model')

In [ ]:
model = ARIMA(medname_logscale, order =(0,1,2))
results_MA = model.fit(disp = -1)
plt.plot(DSLDS)
plt.plot(results_ARIMA.fittedvalues, color ='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-DSLDS['Consumption'])**2))
print('Plotting MA model')

In [ ]:
from pyparsing import results
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)

In [ ]:
predictions_ARIMA_diff_cumsum =predictions_ARIMA_diff_cumsum()


In [ ]:
predictions_ARIMA_log = pd.Series(medname_logscale['Consumption'].ix[0],index = medname_logscale.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value =  0)


In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(medname)
plt.plot(predictions_ARIMA)

In [ ]:
medname_logscale

In [ ]:
results_ARIMA.plot_predict(1,24)
x = results_ARIMA.forecast(steps=12)

In [ ]:
x[1]

In [ ]:
len(x[1])

In [ ]:
np.exp(x[1])